In [7]:
import numpy as np
from sklearn.datasets import make_blobs

from sklearn.model_selection import train_test_split

from scipy.stats import norm


In [14]:
# Creating discrete data
X0 = np.random.randint(2, size = 1000) # Creating X0 column 
X1 = np.random.randint(2, size = 1000) # Creating X1-column
y =np.random.randint(2, size = 1000) # Target/ y column

X = np.stack([X0,X1] , axis = 1) # Using stack function instead of concatenate to add values on a new axis 

In [38]:
class NaiveBayes:
    
    def __init__(self):
        #self.X = X
        #self.y = y
        self.X, self.y = make_blobs(n_samples = 10000, centers = 2, n_features = 2, random_state = 1)
        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(self.X,self.y, random_state= 0,
                                                                               test_size = 0.3)
        
    def fit_dist(self, data):
        mu = np.mean(data)
        sigma = np.std(data)
        
        dist = norm(mu, sigma)
        
        return dist
        
        
    def probability(self, X, prior, dist1, dist2):
        return prior * dist1 * dist2
    
    
    def run_model(self):
        #Separating data according to class to make it easier

        self.X_0_train = self.X_train[self.y_train == 0]
        self.X_1_train = self.X_train[self.y_train == 1]
        
        self.X0_0_train = self.X_0_train[:, 0] # first column X0 and points which belongs to class 0
        self.X1_0_train = self.X_0_train[:, 1] # second column X1 and points which belongs to class 0
        self.X0_1_train = self.X_1_train[:, 0] # first column X0 and points which belongs to class 1
        self.X1_1_train = self.X_1_train[:, 1] # second column and points which belongs to class 1
        
        
        # prior
        self.prior_y0 = len(self.X_0_train) / len(self.X_train) # prior for class 0
        self.prior_y1 = len(self.X_1_train) / len(self.X_train) # prior for class 1

        # Calculating likelihoods by adding laplace smoothing
        lap = 1 # Laplace 1 smoothing
        # Here X takes 2 values - 0 or 1 
        uval = len(np.unique(X))
        # For class 0
        self.X00_0_train = (len(self.X0_0_train[self.X0_0_train == 0]) + lap) / (len(self.X_0_train) + uval*lap)
        self.X01_0_train = (len(self.X0_0_train[self.X0_0_train == 1]) + lap) / (len(self.X_0_train) + uval*lap)
        self.X10_0_train = (len(self.X1_0_train[self.X1_0_train == 0]) + lap) / (len(self.X_0_train) + uval*lap)
        self.X11_0_train = (len(self.X1_0_train[self.X1_0_train == 1]) + lap) / (len(self.X_0_train) + uval*lap)
        
        # For class 1
        self.X00_1_train = (len(self.X0_1_train[self.X0_1_train == 0]) + lap) / (len(self.X_1_train) + uval*lap)
        self.X01_1_train = (len(self.X0_1_train[self.X0_1_train == 1]) + lap) / (len(self.X_1_train) + uval*lap)
        self.X10_1_train = (len(self.X1_1_train[self.X1_1_train == 0]) + lap) / (len(self.X_1_train) + uval*lap)
        self.X11_1_train = (len(self.X1_1_train[self.X1_1_train == 1]) + lap) / (len(self.X_1_train) + uval*lap)        
            
        
    def predict(self):
        
        for sample, target in zip(self.X_test, self.y_test):
            
            # Assigning distributions based on conditions
            # for y = 0 i.e class 0
            if sample[0] == 0:
                self.dist_X0y0 = self.X00_0_train
            else:
                self.dist_X0y0 = self.X01_0_train
                
            if sample[1] == 0:
                self.dist_X1y0 = self.X10_0_train
            else:
                self.dist_X1y0 = self.X11_0_train
        
            if sample[0] == 1:
                self.dist_X0y1 = self.X00_1_train
            else:
                self.dist_X0y1 = self.X01_1_train
                
            if sample[1] == 1:
                self.dist_X1y1 = self.X10_1_train
            else:
                self.dist_X1y1 = self.X11_1_train
                
            
            py0 = self.probability(sample, self.prior_y0, self.dist_X0y0, self.dist_X1y0)
            
            py1 = self.probability(sample, self.prior_y1, self.dist_X0y1, self.dist_X1y1)
            
            
            print('P(y=0| %s) = % 3f' % (sample, py0*100))
            print('P(y=1| %s) = % 3f' % (sample, py1*100))
            
            print('Model predicted class {} and the truth was {} \n'. format(np.argmax([py0*100, py1*100]), target))
        

In [39]:
nb = NaiveBayes()
nb.run_model()

In [40]:
nb.predict()

P(y=0| [-1.50030244  5.11349108]) =  0.000004
P(y=1| [-1.50030244  5.11349108]) =  0.000004
Model predicted class 0 and the truth was 0 

P(y=0| [-9.70304468 -3.64272044]) =  0.000004
P(y=1| [-9.70304468 -3.64272044]) =  0.000004
Model predicted class 0 and the truth was 1 

P(y=0| [-11.12942627  -3.09708926]) =  0.000004
P(y=1| [-11.12942627  -3.09708926]) =  0.000004
Model predicted class 0 and the truth was 1 

P(y=0| [-10.11438309  -5.52374255]) =  0.000004
P(y=1| [-10.11438309  -5.52374255]) =  0.000004
Model predicted class 0 and the truth was 1 

P(y=0| [-0.61779984  4.65345407]) =  0.000004
P(y=1| [-0.61779984  4.65345407]) =  0.000004
Model predicted class 0 and the truth was 0 

P(y=0| [-1.25940623  7.12387611]) =  0.000004
P(y=1| [-1.25940623  7.12387611]) =  0.000004
Model predicted class 0 and the truth was 0 

P(y=0| [-8.30755615 -2.43706693]) =  0.000004
P(y=1| [-8.30755615 -2.43706693]) =  0.000004
Model predicted class 0 and the truth was 1 

P(y=0| [-10.7898651   -4.1